In [13]:
import numpy as np
from qiskit import QuantumCircuit,transpile,assemble
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt

In [14]:
def teleportation_circuit(state_vector):
    # Create a quantum circuit with 3 qubits and 2 classical bits
    circuit = QuantumCircuit(3, 2)

    # Create entangled pair with qubits 1 and 2
    circuit.h(1)
    circuit.cx(1, 2)

    # **Change: Initialize only qubit 0 with the state vector**
    circuit.initialize(state_vector, [0])  # Correctly initializes only qubit 0

    # Barrier for visualization
    circuit.barrier()

    # Bell state measurement
    circuit.cx(0, 1)  # CNOT from qubit 0 to qubit 1
    circuit.h(0)      # Hadamard on qubit 0

    # Barrier for visualization
    circuit.barrier()

    # Measure qubits 0 and 1
    circuit.measure([0, 1], [0, 1])

    # Apply corrections based on measurement outcomes
    circuit.x(2).c_if(circuit.clbits[1], 1)  # Apply X gate if qubit 1 measurement is 1
    circuit.z(2).c_if(circuit.clbits[0], 1)  # Apply Z gate if qubit 0 measurement is 1

    # Visualize the circuit
    print("Quantum Teleportation Circuit:")
    circuit.draw('mpl')

    # Transpile the circuit for the aer_simulator backend
    aer_simulator = Aer.get_backend('aer_simulator')
    transpiled_circuit = transpile(circuit, aer_simulator)
    
    return transpiled_circuit

In [15]:
state_vectors=[[1,0,0,0],
               [0,1,0,0],
               [1/np.sqrt(2),0,0,1/np.sqrt(2)],
               [1/np.sqrt(2),1/np.sqrt(2),0,0],]

In [16]:
for state_vector in state_vectors:
    transpiled_circuit=teleportation_circuit(state_vector)
    print(f"Transpiled Circuit for state{state_vector}:")
    print(transpiled_circuit)

QiskitError: 'StatePreparation parameter vector has 4 elements, therefore expects 2 qubits. However, 1 were provided.'